In [1]:
import os
import glob

import random
import pathlib
import numpy as np
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.patches as patches
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.backends.cudnn.benchmark = True
# import cupy as cp

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import animation
from matplotlib.animation import PillowWriter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors

import time
import multiprocessing as m
import concurrent.futures
from sklearn.metrics import confusion_matrix


import pickle

In [2]:
# def clc():
#     current_dir = os.getcwd()
#     pth_files = glob.glob(os.path.join(current_dir, "*.pth"))
#     for pth_file in pth_files:
#         os.remove(pth_file)

#     pkl_files = glob.glob(os.path.join(current_dir, "*.pkl"))
#     for pkl_file in pkl_files:
#         os.remove(pkl_file)
#     svg_files = glob.glob(os.path.join(current_dir, "*.svg"))
#     for svg_file in svg_files:
#         os.remove(svg_file)

#     print("已清空当前目录下所有.pth & .pkl & .svg 文件")
# clc()

### 注意检查参数！！！！

In [3]:
resample_size = 10 # down sample from 28 to 10
thold = 0.22
pad_size = 0
size = resample_size*resample_size + 2*pad_size

ref_ind = 3.45          # Refractive index of Silicon
free_wv = 1.55e-6       # Wavelength in freespace
wv = free_wv/ref_ind    # Wavelength in Silicon

Hz = 3e8/wv
rect_length = 0.5e-6
layer_length = size*rect_length

z_min = size * rect_length * np.sqrt((4*rect_length**2)/wv**2-1) *1e6
print("确保全连接的最小距离是: " + str(z_min))
z = 100e-6 # 300e-6

num_layers = 1
lr = 0.1
batch_size = 100
epochs = 200

# RAM 64GB,8cpu ,size =100, num_process = 20, W --> 63 sec
# RAM 64GB,8cpu ,size =120, num_process = 20, W --> 129 sec

确保全连接的最小距离是: 99.42603230595124


### Build and train D2NN

In [4]:
def core_matrix(z):
    distance = z                         # distance bewteen two layers (3cm)
    wl = 3e8 / Hz                        # wave length
    wn = 2 * np.pi / wl                  # wave number
    c = rect_length
    w = np.zeros((size, size), dtype=np.complex64)

    for i in range(0, size, 1):
        s = i % size
        for k in range(i+1, size, 1):
            n = k % size
            # print(n-s)
            r_ns = np.sqrt(np.square(distance) + np.square(n * c - s * c))
            w[i][k] = np.exp(1j * wn * r_ns) * (1 / (2 * np.pi * r_ns) + 1 / (1j * wl)) * distance / (r_ns ** 2)
            

    w_diagonal = np.exp(1j * wn * distance) * (1 / (2 * np.pi * distance) + 1 / (1j * wl)) * distance / (distance*distance)
    w = w.T + w

    for i in range(0, size, 1):
        w[i][i] = w_diagonal
    
    w = (w * rect_length * 0.22e-6) 
    return w

start = time.time()
W = core_matrix(z)
end = time.time()
print('NumPy计算时间：', end - start, 's')

print(np.shape(W))
# np.savetxt(r'W.txt', np.array(W),delimiter = ',')

NumPy计算时间： 0.05896759033203125 s
(100, 100)


In [5]:
def convert_to_col_vector(arr):
    vec = arr.view(arr.size(0), -1).t()
    return vec

def convert_to_mat(arr):
    mat = arr.t().view(arr.size(1), size, 1)
    return mat

def detector_region(x): # 输入是 x_intisity
    '''检测区域应该集中在中间部分'''
    i = size/165
    m = 10
    step_sizes = [10, 25, 40, 55, 70, 85, 100, 115, 130, 145]
    result = []
    for step in step_sizes:
        slice = x[:, int(i*step):int(i*(step+m)), :]
        result.append((slice / torch.max(slice)).mean(dim=(1, 2)).unsqueeze(-1))
    return torch.cat(result, dim=-1)
    # 测试过了， 一定要 归一化光强

    
class D2NN(torch.nn.Module):

    def __init__(self, num_layers = num_layers):
        super(D2NN, self).__init__()

        self.phase = torch.nn.Parameter(torch.from_numpy(np.random.random(size=(num_layers, size, 1)).astype('float32')),
                                         requires_grad=True)
    
        # self.sofmax = torch.nn.Softmax(dim = -1)
        self.W = torch.from_numpy(W).cuda()

    def forward(self, x): 
        # 注意： 不需要做 phase reset
        x = torch.complex(x, torch.zeros_like(x)) # 相位图后不需要这一行
        x = convert_to_col_vector(x) 
        P = torch.matmul(self.W, x)
        
        for i in range(self.phase.size(0)):
            phase = torch.exp(1j * self.phase[i].squeeze())
            P = torch.matmul(self.W, P * phase.unsqueeze(-1))
        
        x = convert_to_mat(P)
        x_intisity = torch.abs(x)**2
        output = detector_region(x_intisity) * 10      
        return output
    

class Resize(object):
    def __init__(self, size=10):
        self.size = size

    def __call__(self, image):
        image = np.array(image)
        extracted_image = image[5: 25: 2, 5: 25: 2]
        return extracted_image

# class Resize(object):
#     def __init__(self, size=10):
#         self.size = size

#     def __call__(self, image):
#         image = np.array(image)
#         extracted_image = image[5: 25: 1, 5: 25: 1]
#         return extracted_image
    

training_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
def train_sce(num_layers):
    model_name ='.pth'
    model_save_path = "./mnist_"+str(size)+"_"+str(num_layers)+"_"
    
    # resize 之后还是个2D图， 要 flatten
    transform = transforms.Compose([Resize(size=resample_size),
                                    transforms.ToTensor(),transforms.Lambda(lambda x: torch.flatten(x))])

    train_dataset = torchvision.datasets.MNIST(root = "./data",train=True,transform=transform,download=True)
    val_dataset = torchvision.datasets.MNIST(root = "./data",train=False, transform=transform,download=True)

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=0, shuffle=True, pin_memory=True)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, num_workers=0, shuffle=False, pin_memory=True)

    model = D2NN(num_layers)
    # model.load_state_dict(torch.load(r'./test00_mnist_200_10_54.pth'), strict=False)
    model.cuda()

    # criterion = torch.nn.MSELoss(reduction='sum').cuda()    # MSE loss function
    criterion = torch.nn.CrossEntropyLoss().cuda() # justified
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))  # Adam optimizer
    # adjust_learning_rate = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = epochs, eta_min=0, last_epoch=-1)
    # 余弦退火周期的长度 会影响 acc，可以尝试设置大于 50 的值
    
    adjust_learning_rate = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

    for epoch in range(1,  1 + epochs):  # train
        
        model.train()

        train_len,train_running_counter, train_running_loss  = 0.0, 0.0, 0.0

        tk0 = tqdm(train_dataloader, ncols=100, total=int(len(train_dataloader)))
        for train_iter, train_data_batch in enumerate(tk0):
            
            train_images = train_data_batch[0].cuda()
            a = torch.zeros_like(train_images)
            b = torch.ones_like(train_images)
            train_images = torch.where(train_images >thold, b, a)  # 图像预处理: 转成二值图
            # 一个 batch 中[0]是图片，[1]是标签, 先得到相位图， 再填充 0  
            # train_images = torch.exp(1j*train_data_batch[0].cuda()*2*np.pi) # 相位图
            train_images = F.pad(train_images, pad=(pad_size,pad_size))
            train_labels = F.one_hot(train_data_batch[1].cuda(), num_classes=10).float()   # convert the label to one-hot form
            train_images = train_images.squeeze(dim=1)
            
            train_outputs = model(train_images)
            train_loss_ = criterion(train_outputs, train_labels)

            train_counter_ = torch.eq(torch.argmax(train_labels, dim=1), torch.argmax(train_outputs, dim=1)).float().sum()

            optimizer.zero_grad()
            train_loss_.backward()
            optimizer.step()

            train_len += batch_size
            train_running_loss += train_loss_.item()
            train_running_counter += train_counter_

            train_loss = float(train_running_loss / train_len)
            train_accuracy = float(train_running_counter / train_len)

            tk0.set_postfix({'Train_Accuracy': '{:.4f}'.format(train_accuracy), 'Train_Loss': '{:.4f}'.format(train_loss)})

        training_history['train_loss'].append(train_loss)
        training_history['train_acc'].append(train_accuracy)


        adjust_learning_rate.step()

        
        with torch.no_grad():
            
            model.eval()
            val_len,val_running_counter, val_running_loss = 0.0, 0.0, 0.0

            tk1 = tqdm(val_dataloader, ncols=100, total=int(len(val_dataloader)))
            for val_iter, val_data_batch in enumerate(tk1):
                
                val_images = val_data_batch[0].cuda()
                a = torch.zeros_like(val_images)
                b = torch.ones_like(val_images)
                val_images = torch.where(val_images >thold, b, a)  # 图像预处理: 转成二值图
                # val_images = torch.exp(1j*val_data_batch[0].cuda()*2*np.pi) # phase object
                val_images = F.pad(val_images, pad=(pad_size,pad_size))
                val_labels = F.one_hot(val_data_batch[1].cuda(), num_classes=10).float()

                val_images = val_images.squeeze(dim=1)
                val_outputs = model(val_images)

                val_loss_ = criterion(val_outputs, val_labels)
                val_counter_ = torch.eq(torch.argmax(val_labels, dim=1), torch.argmax(val_outputs, dim=1)).float().sum()

                val_len += batch_size
                val_running_loss += val_loss_.item()
                val_running_counter += val_counter_

                val_loss = float(val_running_loss / val_len)
                val_accuracy = float(val_running_counter / val_len)

                
                tk1.set_postfix({'Val_Accuracy': '{:.4f}'.format(val_accuracy), 'Val_Loss': '{:.4f}'.format(val_loss)})

            training_history['val_loss'].append(val_loss)
            training_history['val_acc'].append(val_accuracy)

        torch.save(model.state_dict(), (model_save_path + str(epoch) + model_name))
        print('Model : "' + model_save_path + str(epoch) + model_name + '" saved.')
        
        with open("training_history"+"_"+str (size) +"_"+ str(num_layers)+".pkl", 'wb') as f:
            pickle.dump(training_history, f)

In [ ]:
train_sce(num_layers)

100%|███████████████████████| 100/100 [00:01<00:00, 53.28it/s, Val_Accuracy=0.5522, Val_Loss=0.0148]


Model : "./mnist_100_1_1.pth" saved.


100%|███████████████████████| 100/100 [00:01<00:00, 52.72it/s, Val_Accuracy=0.5625, Val_Loss=0.0143]


Model : "./mnist_100_1_2.pth" saved.


100%|███████████████████████| 100/100 [00:01<00:00, 52.74it/s, Val_Accuracy=0.5549, Val_Loss=0.0143]


...

100%|███████████████████████| 100/100 [00:01<00:00, 52.22it/s, Val_Accuracy=0.5772, Val_Loss=0.0138]


Model : "./mnist_100_1_136.pth" saved.


100%|███████████████████████| 100/100 [00:01<00:00, 55.34it/s, Val_Accuracy=0.5767, Val_Loss=0.0138]


Model : "./mnist_100_1_137.pth" saved.


 59%|███████████▏       | 354/600 [00:09<00:06, 39.35it/s, Train_Accuracy=0.5718, Train_Loss=0.0141]

### ID2NN模型评估

In [ ]:
with open('training_history_300_5.pkl', 'rb') as f:
    training_history = pickle.load(f)
    
def plot_training_history():
    
    # Create a figure with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    # fig.set_dpi(300)
    # Plot training and validation loss
    epochs = len(training_history['train_loss'])
    axs[0].plot(range(1, epochs+1), [l for l in training_history['train_loss']], label='train_loss')
    axs[0].plot(range(1, epochs+1), [l for l in training_history['val_loss']], label='val_loss')
    axs[0].set_title('Training and validation loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()

    # Plot training and validation accuracy
    axs[1].plot(range(1, epochs+1), [a for a in training_history['train_acc']], label='train_acc')
    axs[1].plot(range(1, epochs+1), [a for a in training_history['val_acc']], label='val_acc')
    axs[1].set_title('Training and validation accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    plt.savefig("./training_history.svg", dpi=600)
    plt.show()
plot_training_history()

np.max(training_history['val_acc']) # 超过 92.6%